In [1]:
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.blockbatch.construct import construct_data_readers
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "telepotr"

# Define the input data range.
readers: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=[model_name],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS,
)


# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0] # use the first reader
)

2025-01-09 16:24:38 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=87387
2025-01-09 16:24:38 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=87387
2025-01-09 16:24:38 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76 process=87387
2025-01-09 16:24:38 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=87387
2025-01-09 16:24:38 [info     ] prepared 22 input batches.     filename=byblock.py lineno=85 process=87387
2025-01-09 16:24:38 [info     ] initialized duckdb at /var/folders/80/jqw6kmgj7j34j0x4mcs6hcl00000gn/T/v_ta4c8j/op-analytics.duck.db filename=client.py lineno=94 process=87387
2025-01-09 16:24:38 [info     ] reading dataset='ingestion/logs_v1' using 1/1 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=87387
2025-01-09 16:24:38 [info     ] readi

In [2]:
from op_analytics.datapipeline.chains.goldsky_chains import goldsky_mainnet_chains_df
from op_analytics.datapipeline.chains.across_bridge import load_across_bridge_addresses

# Fetch across bridge addesses
goldsky_df = goldsky_mainnet_chains_df()
across_bridge_addresses_df = load_across_bridge_addresses(chains_df=goldsky_df)


ctx.client.sql("CREATE TABLE IF NOT EXISTS across_bridge_metadata AS SELECT * FROM across_bridge_addresses_df")
ctx.client.sql("CREATE TABLE IF NOT EXISTS op_stack_chain_metadata AS SELECT * FROM goldsky_df")

2025-01-09 16:24:38 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=87387
2025-01-09 16:24:40 [info     ] Loaded OP chains metadata from /Users/chuxinhuang/work/op-analytics/op_chains_tracking/inputs/chain_metadata_raw.csv filename=load.py lineno=85 process=87387
2025-01-09 16:24:40 [info     ] [REPO vs. GSHEETS] ERROR: Chain Metadata is different filename=load.py lineno=97 process=87387


In [3]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ across_bridge_metadata  │
│ op_stack_chain_metadata │
└─────────────────────────┘

In [4]:
# view = auxiliary_views["telepotr/bridging_transactions"].create_view(
#     duckdb_context=ctx,
#     template_parameters={
#         "raw_logs": input_datasets["ingestion/logs_v1"].as_subquery(),
#         "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
#         "across_bridge_metadata": "across_bridge_metadata",
#         "op_stack_chain_metadata": "op_stack_chain_metadata",
#     },
# )

2025-01-09 16:24:41 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=87387
2025-01-09 16:24:41 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=87387
2025-01-09 16:24:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=87387 template=telepotr/bridging_transactions
2025-01-09 16:24:41 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=87387


In [5]:

rel = auxiliary_views["telepotr/bridging_transactions"].to_relation(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets["ingestion/logs_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
        "across_bridge_metadata": "across_bridge_metadata",
        "op_stack_chain_metadata": "op_stack_chain_metadata",
    },
)

2025-01-09 16:24:41 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=87387
2025-01-09 16:24:41 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=87387
2025-01-09 16:24:41 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=87387 template=telepotr/bridging_transactions


In [6]:
rel

┌────────────┬─────────────────┬──────────────┬───────────┬──────────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────┬─────────────────────┬─────────────────────┬─────────────────┬───────────────┬──────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────┬───────────┬───────────────────────┬───────────────────────┬───────────┬───────────────────────┐
│     dt     │ block_timestamp │ block_number │ src_chain │ src_chain_id │              contract_address              │                          transaction_hash                          │ deposit_id │            input_token_address             │            output_token_address            │ dst_chain_id │    input_amount     │    output_amount  